In [2]:
import numpy as np
import pandas as pd
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
import matplotlib.pyplot as plt
from sklearn.metrics import roc_auc_score, roc_curve
import matplotlib.pyplot as plt
from sklearn.impute import KNNImputer
from sklearn.impute import SimpleImputer
from sklearn.datasets import load_iris
from sklearn.ensemble import GradientBoostingClassifier,ExtraTreesClassifier,AdaBoostClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.ensemble import StackingClassifier


In [3]:
df = pd.read_csv("C:\\Users\Ali\\DataScience_Projects\\Kaggle_Competition\\train_set.csv")
df.head()

,RecordId,X2,X3,X4,X5,X6,X7,X8,X9,X10,...,X70,X71,X72,X73,X74,X75,X76,X77,X78,Y
0,1,87.000000,34.118411,0,2,0,165.100000,1,829,2,...,0.040000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
1,2,82.372284,31.573280,0,0,1,162.983897,1,724,0,...,0.033431,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
2,3,50.000000,27.771653,0,0,1,165.100000,1,895,2,...,0.010000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
3,4,66.236109,26.515922,0,0,1,167.009549,1,637,0,...,0.039363,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
4,5,81.303299,20.843691,0,0,1,158.165419,0,564,0,...,0.069242,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0


In [4]:
X = df.iloc[:, df.columns != 'Y']
y = df[['Y']]

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2)

In [6]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


In [7]:
imr = SimpleImputer(missing_values=np.nan, strategy='mean')
imr = imr.fit(X_train_scaled)
imputed_data = imr.transform(X_train_scaled)
X_train_scaled = pd.DataFrame(imputed_data)

In [8]:
imr = SimpleImputer(missing_values=np.nan, strategy='mean')
imr = imr.fit(X_test_scaled)
imputed_data = imr.transform(X_test_scaled)
X_test_scaled = pd.DataFrame(imputed_data)

In [9]:
pca = PCA(n_components=0.95)
X_train_pca = pca.fit_transform(X_train_scaled)
X_test_pca = pca.transform(X_test_scaled)

In [10]:
model = GradientBoostingClassifier(n_estimators=150, learning_rate=0.1, max_depth=5, random_state=100)
model.fit(X_train_pca,y_train)

c:\Users\Ali\anaconda3\Lib\site-packages\sklearn\ensemble\_gb.py:424: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


GradientBoostingClassifier(max_depth=5, n_estimators=150, random_state=100)

In [11]:
y_pred = model.predict(X_test_pca)

In [12]:
accuracy = accuracy_score(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)
class_report = classification_report(y_test, y_pred)
accuracy

0.9953682072117827

In [13]:
md_predictions_probs = model.predict_proba(X_test_pca)
md_predictions_probs 

array([[9.99680355e-01, 3.19644871e-04],
       [9.98966035e-01, 1.03396531e-03],
       [9.99749376e-01, 2.50623731e-04],
       ...,
       [8.58667603e-01, 1.41332397e-01],
       [9.99791436e-01, 2.08563808e-04],
       [9.99699598e-01, 3.00402481e-04]])

In [19]:
test_data = pd.read_csv("C:\\Users\Ali\\DataScience_Projects\\Kaggle_Competition\\test_set.csv")

In [15]:
recordId = test_data['RecordId']
test_data = test_data.drop(columns = 'RecordId')

In [20]:
scaler = StandardScaler()
test_data = scaler.fit_transform(test_data)
test_data = scaler.transform(test_data)

c:\Users\Ali\anaconda3\Lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


In [21]:
imr = SimpleImputer(missing_values=np.nan, strategy='mean')
imr = imr.fit(test_data)
imputed_data = imr.transform(test_data)
test_data = pd.DataFrame(imputed_data)

In [22]:
test_data = pca.transform(test_data)

In [23]:
prediction = model.predict_proba(test_data)
prediction= prediction[:,1]

In [24]:
finaldf = pd.concat([recordId, pd.DataFrame(prediction, columns=['Y'])], axis=1)
finaldf
finaldf.to_csv("C:\\Users\Ali\\DataScience_Projects\\Kaggle_Competition\\PCA_Gradient_test_results.csv", index= False)